In [1]:
# DEPENDENCIES
import glob
from shutil import copyfile
import pickle
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator, FixedLocator)
import logomaker
# LOCAL IMPORTS
from indexing_classes import GPCRDBIndexing
from gain_classes import GainDomain, GainCollection, Anchors, GPS
import sse_func


In [2]:
def filter_by_receptor(sequences, selection):
    new_list = []
    for seq_tup in sequences:
        if selection in seq_tup[0]:
            new_list.append(seq_tup)
    return new_list

def filter_by_list(sequences, selection): # selection list
    new_list = []
    for seq_tup in sequences:
        for it in selection:
            if it in seq_tup[0]:
                new_list.append(seq_tup)
    return new_list

In [3]:
enum = 153
valid_seqs = sse_func.read_multi_seq("/home/hildilab/projects/agpcr_nom/app_gain_gain.fa") #app_gain_gain.fa # app_gain_001_gain.fa
print(len(valid_seqs))
quality_file = "/home/hildilab/projects/agpcr_nom/app_gain_gain.mafft.jal"
#quality_file = "/home/hildilab/projects/agpcr_nom/all001.jal"
#quality_file = f"/home/hildilab/projects/agpcr_nom/app_gain_gain_mafft.{enum}.jal"
alignment_file = "/home/hildilab/projects/agpcr_nom/app_gain_gain.mafft.fa"
#alignment_file = "/home/hildilab/projects/agpcr_nom/all001.fa"
#alignment_file = f"/home/hildilab/projects/agpcr_nom/app_gain_gain_mafft.{enum}.fa"
#stride_folder = "/home/hildilab/projects/agpcr_nom/all_gps_stride"
#stride_files = glob.glob("/home/hildilab/projects/agpcr_nom/all_gps_stride/*")
stride_files = glob.glob("/home/hildilab/projects/agpcr_nom/sigmas/sigma_2/*")
# This only contains the sigma files for truncated (?) PDBs.
quality = sse_func.read_quality(quality_file)
# -1 of the ACTUAL COLUMN (6782) in JALVIEW since there is is ONE-INDEXED
gps_minus_one = 6781 # default: 6781; 160: 6607; 155: 6719
aln_cutoff = 6826 # default: 6826; 160: 6645; 155: 6765
#gps_minus_one = 5247 # for the 0.01 constructed set
#aln_cutoff = 5288 # for the 0.01 constructed set
alignment_dict = sse_func.read_alignment(alignment_file, aln_cutoff)

#anchors = [ 662, 1194, 1912, 2490, 2848, 3011, 3073, 3260, #H1-H8
#            3455, 3607, 3998, 4279, 4850, 5339, #5341 S1-S6, S7 REMOVED!
#            5413, 5813, 6337, 6659, 6696, 6765, 6808] #S8-13
#anchor_occupation = [ 4594.,  6539., 11392., 13658.,  8862., 5092.,  3228., 14189., #H1-H8
#                      9413., 12760.,  9420., 11201., 12283., 3676.,#  4562. S1-S6, S7 REMOVED!
#                     13992., 12575., 13999., 14051., 14353., 9760., 14215.] #S8-13
#anchor_dict = sse_func.make_anchor_dict(anchors, 3425)

14435


In [4]:
# re-offset the sequences to match the exact PDB indexing
full_seqs = sse_func.read_alignment("../full_sequences_23.fa")
valid_adj_seqs = []
for tup in valid_seqs:
    x = sse_func.find_the_start(longseq=full_seqs[tup[0]], shortseq=tup[1])
    if x == 0:
        print("already 1st res.\n", tup[0], tup[1][:10], tup[1][-10:])
        valid_adj_seqs.append( (tup[0],full_seqs[tup[0]][:len(tup[1])-1]) )
        print(full_seqs[tup[0]][:len(tup[1])-1])
    else:
        valid_adj_seqs.append( (tup[0],full_seqs[tup[0]][x-1:x+len(tup[1])-1]) )
              
"""with open("offset_valid_seqs.fa","w") as fa:
    for tup in valid_adj_seqs:
        fa.write(f">{tup[0]}\n{tup[1]}\n")"""


already 1st res.
 A0A2B4RIV8-A0A2B4RIV8_STYPI-CadherinEGFLAGseven-passG-typereceptor1-Stylophora_pistillata MIRKSIFNAA LTHFAVLVDY
MIRKSIFNAAIALEEFALNYGKNNLTDGNSSVKIDSHKLVLVIQKADRENAGEFYPRNQELQATMKLSCASFVNNGIIVNKVSVVVGIFYKDLHEILFIDQSFTIQTGNTRYLDSGIMAMAMDPKPEKLQENVILNFRNLNVHEGERICMFWSGLNESSGGFSETGCYEVASESNSEETVCSCDHLTHFAVLVD
already 1st res.
 A0A7J7K268-A0A7J7K268_BUGNE-CELSR1-Bugula_neritina MPNTSNSEDD HMTSFAIMME
MPNTSNSEDDLRQQLELFESVKYFEVDDVIDPREADLSMQHLVTVVNDLLDADTWKDAWDTIDDNSTRGPVALMTDLEELAMKITKNLYIGLDKEDDCSNFKSGAENLLMSCQKRRLKLCDSEFIFPEYDKLSADPSLNSTLEKWKRPTDNVYMPASLFPKHEYPDDAVVSVAVFLYEEASSVIPSRLDVSTFINLEATTSADNPEAAVDPDKVEQSTVNSRIMSILVDPQVPAPLDECIIITLEHKQANKDNKRTSYCAFWDFKLAGMLGGGWSSDGCEVAETNETHTVCCCYHMTSFAIMM
already 1st res.
 A0A1A8MYS8-A0A1A8MYS8_9TELE-Cadherin,EGFLAGseven-passG-typereceptor3-Nothobranchius_pienaari. GRLYGNDLQI LGTFGVLMDS
GRLYGNDLQIAERLLSRLLTFETQQSGFGLTATQDAHFNENILRGCSVLMGPGTSGLWRALAQSQNHILGGGPAELTELLEQYSKNLAQNMKLTYLNPVALVSPNIVMNLDRVENLTYVRQRFPRYHATMFRGQTLWDPYTHVTLP

'with open("offset_valid_seqs.fa","w") as fa:\n    for tup in valid_adj_seqs:\n        fa.write(f">{tup[0]}\n{tup[1]}\n")'

In [5]:
alignment_file = "../offset_valid_seqs.mafft.fa"
quality_file = "/home/hildilab/projects/agpcr_nom/offset_valid_seqs.jal"
gps_minus_one = 6553 # default: 6781; 160: 6607; 155: 6719
aln_cutoff = 6567 # default: 6826; 160: 6645; 155: 6765
alignment_dict = sse_func.read_alignment(alignment_file, aln_cutoff)
quality = sse_func.read_quality(quality_file)

valid_collection = GainCollection(  alignment_file = alignment_file,
                                    aln_cutoff = aln_cutoff,
                                    quality = quality,
                                    gps_index = gps_minus_one,
                                    stride_files = stride_files,
                                    sequence_files=None,
                                    sequences=valid_adj_seqs,#valid_seqs,
                                    alignment_dict = alignment_dict,
                                    is_truncated = True,
                                    coil_weight=0.00, # TESTING
                                    #domain_threshold=20, # TESTING
                                    stride_outlier_mode=True,
                                    debug=False)

[NOTE] GainCollection.__init__: Found sequences object.
[WARNING] sse_func.detect_GPS: GPS-1 column is empty. Returning empty for alternative Detection.
	gps_minus_one  = 6553
	alignment_indices[-15:] = array([6466, 6472, 6473, 6474, 6482, 6522, 6523, 6531, 6536, 6541, 6542,
       6543, 6545, 6546, 6547])
[DEBUG] gain_classes.GPS : ALTERNATIVE GPS DETECTION! DEBUG:last_sheets = [(453, 458), (463, 466)]
Info about this GPS:
self.isConsensus = False
self.indices = [332, 333, 334, 335]
self.sequence = array(['N', 'H', 'M', 'T'], dtype='<U1')
self.alignment_indices = array([6531, 6536, 6541, 6542])
self.residue_numbers = [459, 460, 461, 462]
[WARNING] sse_func.detect_GPS: GPS-1 column is empty. Returning empty for alternative Detection.
	gps_minus_one  = 6553
	alignment_indices[-15:] = array([6536, 6537, 6538, 6541, 6542, 6543, 6545, 6546, 6547, 6548, 6549,
       6550, 6551, 6555, 6556])
[DEBUG] gain_classes.GPS : ALTERNATIVE GPS DETECTION! DEBUG:last_sheets = [(469, 471), (474, 475)]
In

In [6]:
with open('../valid_collection.pkl', 'wb') as savefile:
    pickle.dump(valid_collection, savefile)
iehngtfeh

NameError: name 'iehngtfeh' is not defined

In [ ]:
def truncate_pdb(start, end, oldpdb, newpdb):
    with open(oldpdb) as p:
        data = p.readlines()
    newdata = []
    for line in data:
        if line.startswith('ATOM'):
            resid = int(line[22:26])
            if start > resid or end < resid:
                continue
        newdata.append(line)
    with open(newpdb, 'w') as new:
        new.write(''.join(newdata))

pdbs = glob.glob("/home/hildilab/projects/agpcr_nom/*output/**/*_rank_1_*.pdb")
print(f"Found {len(pdbs)} best ranked models in target directories.")
valid_ct = 0

for gain in valid_collection.collection:
    valid_ct +=1
    name = gain.name
    tar_pdb = [p for p in pdbs if name.split("_")[0] in p][0]
    new_pdb_path = f'../all_pdbs/{gain.name}.pdb'
    truncate_pdb(gain.start,gain.end, tar_pdb, new_pdb_path)
print("Copied and truncated", valid_ct, "GAIN domains.")
    

In [ ]:
"""receptor_list = ["AGRA2","AGRA3", "AGRB1", "AGRB2", "AGRB3", "AGRC1", "AGRC2", "AGRC3", "AGRD1", 
                "AGRD2", "AGRE1", "AGRE2", "AGRE3", "AGRE4", "AGRE5", "AGRF1", "AGRF2", "AGRF3", 
                "AGRF4", "AGRF5", "AGRG1", "AGRG2", "AGRG3", "AGRG4", "AGRG5", "AGRG6", "AGRG7", 
                "AGRL1", "AGRL2", "AGRL3", "AGRL4", "AGRV1", "AGRA", "AGRB", "AGRC", "AGRD", 
                "AGRE", "AGRF", "AGRG", "AGRL", "AGRV", "_" ]
                #["EGFLAGseven-passG-typereceptor1", "CELR1", "CELSR1"], 
                #["EGFLAGseven-passG-typereceptor2", "CELR2", "CELSR2"]
                #["EGFLAGseven-passG-typereceptor3", "CELR3", "CELSR3"]]
                 
receptor_list = [["EGFLAGseven-passG-typereceptor1", "CELR1", "CELSR1"], 
                 ["EGFLAGseven-passG-typereceptor2", "CELR2", "CELSR2"],
                 ["EGFLAGseven-passG-typereceptor3", "CELR3", "CELSR3"],
                 ["EGFLAGseven-passG-typereceptor", "CELR", "CELSR"]]
out_names = ("ADGRC1", "ADGRC2", "ADGRC3", "ADGRC")
for i,group in enumerate(receptor_list):
    parse_string = group
    
    print(parse_string)
    filtered_sequences = filter_by_receptor(valid_seqs, parse_string)
    #filtered_sequences = filter_by_list(valid_seqs, group)
    if len(filtered_sequences) == 0:
        continue
    print(f"Parsed with {parse_string = }: Result : {len(filtered_sequences)} Sequences")
    parsed_collection = GainCollection( alignment_file = alignment_file,
                                        aln_cutoff = aln_cutoff,
                                        quality = quality,
                                        gps_index = gps_minus_one,
                                        stride_files = stride_files,
                                        sequence_files=None,
                                        sequences=filtered_sequences,
                                        alignment_dict = alignment_dict,
                                        is_truncated = True,
                                        coil_weight=0.08 # testing
                                         )
    parsed_collection.plot_sse_hist(title=f"Receptor group: {parse_string} (Total: {len(filtered_sequences)})",
                                   n_max=14,
                                   #savename="../fig/hists/%s.adj"%(out_names[i]))
                                   savename="../fig/hists/%s"%(str(group)))"""

In [ ]:
"""    parse_string = "_"
    print(parse_string)
    filtered_sequences = filter_by_receptor(valid_seqs, parse_string)
    #if len(filtered_sequences) == 0:
    print(f"Parsed with {parse_string = }: Result : {len(filtered_sequences)} Sequences")
    parsed_collection = GainCollection( alignment_file = alignment_file,
                                        aln_cutoff = aln_cutoff,
                                        quality = quality,
                                        gps_index = gps_minus_one,
                                        stride_files = stride_files,
                                        sequence_files=None,
                                        sequences=filtered_sequences,
                                        alignment_dict = alignment_dict,
                                        is_truncated = True
                                         )
    parsed_collection.plot_sse_hist(title=f"Receptor group: {parse_string} (Total: {len(filtered_sequences)})",
                                   n_max=16,
                                   savename="hists/%s"%(str(group)))"""

In [ ]:
# Function for Parsing out specific Files from the overall dataset based on selection
def grab_selection(parse_string, stride_path, pdb_list, sequences, profile_path, target_dir, seqs=None):
    # grabs PDB file, stride file, profiles, sequence from FASTA and copies to target dir.
    if seqs is None:
        sub_seqs = [seq for seq in sequences if parse_string.lower() in seq[0].lower()]
    else: sub_seqs = seqs
    print(f"Found {len(sub_seqs)} sequences.")
    strides = glob.glob(stride_path+"*.stride")#
    profiles = glob.glob(profile_path+"*.png")
    
    sub_strides = []
    sub_profiles = []
    sub_pdbs = []
    
    for seq in sub_seqs:
        ac = seq[0].split("-")[0]
        [sub_profiles.append(prof) for prof in profiles if ac in prof]
        [sub_strides.append(stride) for stride in strides if ac in stride]
        [sub_pdbs.append(pdb) for pdb in pdb_list if ac in pdb]
    
    for prof in sub_profiles:
        name = prof.split("/")[-1]
        copyfile(prof, target_dir+"profiles/"+name)
    
    for stride in sub_strides:
        name = stride.split("/")[-1]
        copyfile(stride, target_dir+"strides/"+name)
    
    for pdb in sub_pdbs:
        name = pdb.split("/")[-1]
        copyfile(pdb, target_dir+"pdbs/"+name)
        
    for seq in sub_seqs:
        sse_func.write2fasta(seq[1]+"\n", seq[0], target_dir+"seqs/"+seq[0]+".fa")
        
    print(f"Copied {len(sub_pdbs)} PDB files, {len(sub_strides)} STRIDE files,",
          f" {len(sub_profiles)} Profiles and {len(sub_seqs)} Sequences",
          f"for Selection {parse_string}")
    
root_path = "/home/hildilab/projects/agpcr_nom/"
profile_path = root_path+"all_gps_profiles/"
pdb_list = glob.glob(f"{root_path}all_gps*/batch*/*rank_1_*.pdb")
print(len(pdb_list))
#valid_seqs
target_dir = root_path+"human/"

In [ ]:
"""grab_selection(parse_string='HUMAN',
              stride_path = root_path+"all_gps_stride/",
              pdb_list = pdb_list,
              sequences = valid_seqs,
              profile_path = profile_path,
              target_dir = target_dir)"""

In [ ]:
human_seqs = ["Q9HBW9","O60241","Q6QNK2","Q9UHX3","Q5T601","Q96PE1","O60242","Q86SQ4",
                "O94910","Q8IWK6","Q8IZP9","Q8WXG9","Q86Y34","O95490","Q14246","Q9BY15",
                "Q8IZF2","Q86SQ3","Q8IZF6","Q96K78","Q8IZF3","Q8IZF7","Q8IZF5","Q7Z7M1",
                "Q8IZF4","Q9HCU4","Q9NYQ6","Q9NYQ7","Q9HAR2","O14514","P48960",
                "Q9Y653"]
sigma_2_strides = glob.glob("/home/hildilab/projects/agpcr_nom/sigma_2*/*.stride")
list_32 = filter_by_list(valid_seqs, human_seqs)

human_collection = GainCollection( alignment_file = alignment_file,
                                        aln_cutoff = aln_cutoff,
                                        quality = quality,
                                        gps_index = gps_minus_one,
                                        stride_files =  sigma_2_strides, #stride_files,
                                        sequence_files=None,
                                        sequences=list_32,
                                        alignment_dict = alignment_dict,
                                        is_truncated = True,
                                        coil_weight = 0.08, # TESTING
                                        stride_outlier_mode = True
                                         )
#print(len(human_collection.collection))
"""for gain in human_collection.collection:
    #print(gain.name, gain.start, gain.end, gain.sequence, gain.index, gain.subdomain_boundary)
    pdb_out = root_path+"human/trunc_pdbs/"+gain.name+"_gain.pdb"
    ac = gain.name.split("-")[0]
    found_pdb = [pdb for pdb in pdb_list if ac in pdb]
    target_pdb = found_pdb[0]
    gain.write_gain_pdb(target_pdb, pdb_out)"""


In [ ]:
"""for gain in human_collection.collection:
    if "Q6QNK2" in gain.name:
        print("yea")
        gain.plot_helicality(savename="d1_helicality.svg")
        gain.plot_profile(savename='d1_profile.svg')
        print(gain.subdomain_boundary, gain.start, gain.end)"""
    #print(gain.name, gain.start, gain.end, gain.sequence, gain.index, gain.subdomain_boundary)
    #pdb_out = root_path+"human_32/trunc_pdbs/"+gain.name+"_gain.pdb"
    #ac = gain.name.split("-")[0]
    #found_pdb = [pdb for pdb in pdb_list if ac in pdb]
    #target_pdb = found_pdb[0]
    #gain.write_gain_pdb(target_pdb, pdb_out)


In [ ]:
"""plt.plot(human_collection.anchor_hist)
human_collection.plot_sse_hist(title=f"Receptor group: HUMAN_31 (Total: 31)",
                                   n_max=16,
                                   savename=f"hists/human_31.s2_newAnch.{enum}")"""

In [ ]:
"""#dir(valid_collection)
occupancy = np.zeros([aln_cutoff],dtype=int)
sse_matrix = np.zeros([len(valid_collection.collection), aln_cutoff])
for i, gain in enumerate(valid_collection.collection):
    #print(gain.sda_helices, gain.sdb_sheets)
#    for res_id in range(gain.start,gain.end+1):
    occupancy[gain.alignment_indices] += 1
    
    for helix in gain.sda_helices:
        for res_id in range(helix[0],helix[1]+1):
            sse_matrix[i,gain.alignment_indices[res_id]] = -1
    for sheet in gain.sdb_sheets:
        for res_id in range(sheet[0],sheet[1]+1):
            sse_matrix[i,gain.alignment_indices[res_id]] = 1

anchors , anchor_occupation = valid_collection.find_anchors(cutoff=3000)
anchor_dict = sse_func.make_anchor_dict(anchors, valid_collection.alignment_subdomain_boundary)
#dir(human_collection.collection[0])
#human_collection.collection[0].create_indexing(anchors, anchor_occupation, anchor_dict)
print(anchor_dict)"""

In [ ]:
#print(anchors, anchor_occupation, valid_collection.alignment_subdomain_boundary)
# 000 + 000 ADJUSTED AGAINST S6
#anchors = [ 662, 1194, 1912, 2490, 2848, 3011, 3073, 3260, 3455, 3607, 3998, 4279, 4850, 5341,
# 5413, 5813, 6337, 6659, 6696, 6765, 6808] # removed S6 @  5339
#anchor_occupation = [ 4594.,  6539., 11392., 13658.,  8862., 5092.,  3228., 14189.,  9413., 12760.,
#  9420., 11201., 12283.,  4562.,  13992., 12575., 13999., 14051., 14353., 9760., 14215.] # removed S6 @ ,3676.,
# 000 + 000 ADJUSTED AGAINST S7
anchors = [ 662, 1194, 1912, 2490, 2848, 3011, 3073, 3260, 3455, 3607, 3998, 4279, 4850, 5339,
 5413, 5813, 6337, 6659, 6696, 6765, 6808] # removed S7 @  5341
anchor_occupation = [ 4594.,  6539., 11392., 13658.,  8862., 5092.,  3228., 14189.,  9413., 12760.,
  9420., 11201., 12283.,  3676.,   13992., 12575., 13999., 14051., 14353., 9760., 14215.] # removed S7 @ 4562.,
# 000 + 001 ADJUSTED
# anchors = [ 662, 1194, 1912, 2490, 2848, 2982, 3011, 3073, 3260, 3455, 3607, 3998, 4279, 4850,
#  5341, 5413, 5813, 6337, 6659, 6696, 6765, 6808] # removed S6 @ 5339
# anchor_odccupation = [ 3866.,  6379., 11353., 13614.,  8346.,  3109.,  4990.,  3313., 14135.,  9137.,
# 12558.,  9407., 10876., 12246.,   3921., 13287., 11921., 13976., 14016., 13335., 8890., 14171.] # removed S6 @  3592.,
# 001 ADJUSTED
#anchors = [ 302,  696, 1114, 1409, 1520, 1630, 1916, 2150, 2293, 2839, 2947, 3515, 3995,
# 4070, 4344, 4809, 5137, 5172, 5233, 5275] # removed 5th Helix anchor @1473
#anchor_occupation = [10180., 12340., 12657.,  8731.,  4467.,  4250., 13886., 10035., 12556.,
# 10593., 3416., 13734.,  5258., 13976., 13628., 13977., 14032., 14328.,  9736., 14194.] # removed 5th Helix anchor quality @ 5274
#anchor_dict = sse_func.make_anchor_dict(anchors, valid_collection.alignment_subdomain_boundary)
anchor_dict = sse_func.make_anchor_dict(anchors, 3425)
print(anchor_dict)

In [ ]:
"""plt.figure(figsize=[math.ceil(aln_cutoff/600)+1,4])
plt.subplot(3,1,1)
plt.vlines(valid_collection.alignment_subdomain_boundary,-10000,10000, linewidth=3, color='k')
plt.plot(np.sum(sse_matrix, axis=0))
plt.subplot(3,1,2)
plt.vlines(valid_collection.alignment_subdomain_boundary, -50000,100000, linewidth=3, color='k')
plt.plot(np.convolve(np.sum(sse_matrix, axis=0), np.ones([20])))
plt.subplot(3,1,3)
plt.vlines(valid_collection.alignment_subdomain_boundary, 0, 13000, linewidth=3, color='k')
plt.bar(np.arange(aln_cutoff),valid_collection.anchor_hist, width=2)

for i, anchor in enumerate(anchors):
    print(i, anchor)
    if anchor < valid_collection.alignment_subdomain_boundary: 
        color = u'#1f77b4'
    else: 
        color = u'#ff7f0e'
    plt.scatter(anchor, valid_collection.anchor_hist[anchor]+1000, c=color, marker="1",s=60)
#plt.savefig(f"000.big_signal.s2.mod.png", dpi=600, bbox_inches='tight')"""

In [ ]:

#offset information
def find_offsets(fasta_file, accessions, sequences):
    # searches through the accessions in the big sequence file,
    # finds the start for the provided sequence
    with open(fasta_file,"r") as fa:
        fa_data = fa.read()
        fasta_entries = fa_data.split(">")
    seqs = []
    headers = []
    offsets = []
    for seq in fasta_entries:
        # Fallback for too short sequences
        if len(seq) < 10: 
            continue
        data = seq.strip().split("\n")
        headers.append(data[0].split("|")[1]) # This is only the UniProtKB Accession Number and will be matched EXACTLY
        seqs.append("".join(data[1:]))
    
    heads = np.array(headers)
    for idx, accession in enumerate(accessions):
        #print(np.where(heads == accession))
        seq_idx = np.where(heads == accession)[0][0]
        offset = sse_func.find_the_start(seqs[seq_idx], sequences[idx])
        #print(offset)
        offsets.append(offset)
    
    return offsets

In [ ]:
for gain in human_collection.collection:
    if "Q86Y34" in gain.name:
        print(anchors, anchor_dict)
        x,y,u,a = gain.create_indexing(silent=False,anchors=anchors, anchor_occupation=anchor_occupation, anchor_dict=anchor_dict, debug=True)
        print(x,"\n",y,"\n",u,"\n",a)
        print(gain.sse_dict)
        print(gain.sda_helices)
        print(gain.sdb_sheets)
        
        print(gain.start)
        print(gain.end)
        print(len(gain.alignment_indices))

In [ ]:
mode = 'double'

human_accessions = [gain.name[:6] for gain in human_collection.collection]
human_gain_seqs = [gain.sequence for gain in human_collection.collection]
print(human_accessions)
#for gain in human_collection.collection:
#    print(gain.name,"\n",gain.a_breaks,"\n",gain.b_breaks,"\n","_"*70)

big_seq_file = "/home/hildilab/projects/GPS_massif/uniprot_query/agpcr_celsr.fasta"
human_base = GPCRDBIndexing(human_collection, split_mode=mode, anchors=anchors, anchor_occupation=anchor_occupation, anchor_dict=anchor_dict)
raw_offsets = find_offsets(big_seq_file, human_accessions, human_gain_seqs)
refined_offsets = []
for j, gain in enumerate(human_collection.collection):
    refined_offsets.append(raw_offsets[j]-gain.start)
print(raw_offsets)
print(refined_offsets)


fasta_offsets = find_offsets(big_seq_file, 
                                 human_base.accessions, 
                                 human_base.sequences)
print(f"{fasta_offsets =}")
fa_human_base = GPCRDBIndexing(human_collection, fasta_offsets = fasta_offsets, split_mode=mode, anchors=anchors, anchor_occupation=anchor_occupation, anchor_dict=anchor_dict)
#print(fa_human_base.data2csv)
human_base.data2csv(f"default_indexed_8-13_gpcrdb.csv")
fa_human_base.data2csv(f"uniprot_indexed_8-13_gpcrdb.csv")

In [ ]:
all_base = GPCRDBIndexing(valid_collection, anchors, anchor_occupation, anchor_dict, split_mode='double')
fasta_offsets = find_offsets(big_seq_file, 
                                 all_base.accessions, 
                                 all_base.sequences)
all_base.data2csv(f"default_indexed_all.813.csv")

fa_all_base = GPCRDBIndexing(valid_collection, fasta_offsets = fasta_offsets,  anchors=anchors, anchor_occupation=anchor_occupation, anchor_dict=anchor_dict, split_mode=mode)

fa_all_base.data2csv(f"uniprot_indexed_all.813.csv")

#for i, offset in enumerate(fa_human_base.fasta_offsets):
#    sub_pdb = [pdb for pdb in in_pdbs if fa_human_base.accessions[i] in pdb][0]
#    offset_pdb(sub_pdb, sub_pdb.replace("trunc_pdbs","uniprot_indexed_pdbs"), offset)"""

In [ ]:
def offset_pdb(in_pdb, out_pdb, offset):
    with open(in_pdb,"r") as p:
        data = p.readlines()
    offset_pdb = open(out_pdb, "w")
    
    for line in data:
        if line.startswith("ATOM"):
            res_id = int(line[22:26])
            offset_pdb.write(f"{line[:22]}{str(res_id+offset).rjust(4)}{line[26:]}")
        else:
            offset_pdb.write(line)
    
    print(f"Created PDB {out_pdb} with last residue {res_id+offset}. Total offset {offset} .")

print(len(all_base.unindexed), np.unique(np.array(all_base.unindexed), return_counts=True))

With the "all_base" Indexing and the "validCollection" Collection, we can query the data to give us statistics about the anchors and individual SSE composition.

In [ ]:
#in_pdbs = glob.glob("/home/hildilab/projects/agpcr_nom/human_31/aligned/subd_B/trunc_pdbs/*.pdb")
# For each SSE, get the number of structures having this anchor and stats of which residue is the .50 anchor (via np.where -> anchor)
# Get statistics of the length of each SSE
# Get statistics of loop lengths.
# This may also be subqueued to families...
"""print(dir(all_base))
print(all_base.indexing_dirs[0])
print(valid_collection.collection[0].alignment_indices)
print(len(valid_collection.collection[0].sequence))
print(anchor_dict)"""

def get_loops(indexing_dir):
    # Returns a named dict with loop lengths, i.e. {"H1-H2":13, "H8-S1":12}
    inverted_dir = {sse[0] : (sse[1],ki) for ki, sse in indexing_dir.items()} # The begin of each sse is here {0:(13, "H2")}
    loop_dir = {}
    ordered_starts = sorted(inverted_dir.keys())
    for i, sse_start in enumerate(ordered_starts):
        if i == 0: 
            continue # Skip the first and go from the second SSE onwards, looking in N-terminal direction.
        c_label = inverted_dir[sse_start][1]
        n_end, n_label = inverted_dir[ordered_starts[i-1]]
        loop_dir[f"{n_label}-{c_label}"] = sse_start - n_end - 1
    return loop_dir

def get_sse_len(indexing_dir, total_keys):
    # Returns a dict with the length of each SSE in respective GAIN domain.
    len_dir = {x:0 for x in total_keys}
    for ki in indexing_dir.keys():
        start = indexing_dir[ki][0]
        end = indexing_dir[ki][1]
        len_dir[ki] = end - start + 1
    return len_dir

def get_pos_res(pos_dir, gain):
    # Returns a dict with the One-Letter-Code of each SSE position in the respective GAIN domain.
    pos_res = {k : gain.sequence[pos_dir[k]] for k in pos_dir.keys()}
    return pos_res

def match_dirs(single_dir, collection_dir, exclude=[]):
    for k, v in single_dir.items():
        if v in exclude:
            continue
        if k not in collection_dir.keys():
            collection_dir[k] = [v]
            continue
        collection_dir[k].append(v)
    return collection_dir

seqs = [gain.sequence for gain in valid_collection.collection]

# Test with the first Gain domain of the collection:
print(get_loops(all_base.indexing_dirs[0]))
print(get_sse_len(all_base.indexing_dirs[0], all_base.total_keys))
print(get_pos_res(all_base.center_dirs[0], valid_collection.collection[0]))



In [ ]:
from scipy import stats
#print(dir(all_base))
newkeys = ['H1','H2','H3','H4','H5','H6','H7','H8','S1','S2','S3','S4','S5','S6','S7','S8','S9','S10','S11','S12','S13']
loop_lengths = {}
sse_lengths = {}
center_residues = {}
sse_matrix = np.zeros(shape=(len(all_base.total_keys),len(all_base.total_keys)))
for idx in range(all_base.length):
    #Sanity Check - Do the identifiers match? Yes, they do.
    #if all_base.names[idx].split("-")[0] != valid_collection.collection[idx].name.split("-")[0]:
    #    print(all_base.names[idx].split("-")[0], valid_collection.collection[idx].name.split("-")[0])
    #    raise IndexError
    loop_lengths = match_dirs(get_loops(all_base.indexing_dirs[idx]), loop_lengths)
    sse_lengths = match_dirs(get_sse_len(all_base.indexing_dirs[idx], all_base.total_keys), sse_lengths, exclude=[0])
    center_res = match_dirs(get_pos_res(all_base.center_dirs[idx], valid_collection.collection[idx]), center_residues)

    present_sse = all_base.indexing_dirs[idx].keys()
    for i, kk in enumerate(newkeys):
        for j in range(i,len(newkeys)):
            if kk in present_sse and newkeys[j] in present_sse:
                sse_matrix[j,i] += 1

#plt.imshow(sse_matrix, cmap='gist_yarg')
#plt.xticks(ticks= range(len(newkeys)), labels=newkeys, rotation=90)
#plt.yticks(ticks= range(len(newkeys)), labels=newkeys)
#plt.xlim(-0.5,20.5)
#plt.ylim(20.5,-0.5)
#cbar = plt.colorbar(shrink=0.5)
#plt.savefig("occ_map.svg")
#print(f"\tHas H4 and H5: {len(has4and5)}\n\tHas H5 and H6: {len(has5and6)}\n\tHas H4 and H6: {len(has4and6)}\n\tHas H4,5,6: {len(has_all)}")

In [ ]:
def plot_hist(datarow, color, name, length):
    max = np.max(datarow)
    try: 
        dens = stats.gaussian_kde(datarow)
    except:
        print(np.unique(datarow))
        return
    fig = plt.figure(figsize=[4,2])
    fig.set_facecolor('w')
    n, x, _ = plt.hist(datarow, bins=np.linspace(0,max,max+1), histtype=u'step', density=True, color='white',alpha=0)
    plt.plot(x, dens(x),linewidth=2,color=color,alpha=1)
    plt.fill_between(x,dens(x), color=color,alpha=0.1)
    ax = plt.gca()
    ymax = ax.get_ylim()[1]
    val_string = f'{round(np.average(datarow),2)}±{round(np.std(datarow),2)}'
    plt.text(max, ymax*0.95, name, horizontalalignment='right', fontsize=14, verticalalignment='top')
    plt.text(max, ymax*0.8, val_string, horizontalalignment='right', fontsize=14, verticalalignment='top')
    plt.text(max, ymax*0.65, f"{round(len(datarow)/length*100, 1)}%", horizontalalignment='right', fontsize=14, verticalalignment='top')
    plt.xlabel('Element Length [Residues]')
    plt.ylabel('Relative density [AU]')
    plt.savefig(f'{name}_hist.svg')
    plt.show()
    plt.close(fig)

def parse_conservation(datarow, length):
    total = len(datarow)
    letters, counts = np.unique(np.array(datarow), return_counts=True)

    resid_counts = {}
    for i, res in enumerate(letters):
            resid_counts[int(counts[i])] = res
    
    sorted_counts = sorted(resid_counts.keys())[::-1]

    occupancy = round(total/length*100, 1)
    conserv_string = []
    residue_occupancies = [ int( x*100 / total ) for x in sorted_counts]
    for idx, occ in enumerate(residue_occupancies):
        if occ >= 5: conserv_string.append(f"{resid_counts[sorted_counts[idx]]}:{occ}%")

    return occupancy, ", ".join(conserv_string)
# HISTOGRAMS FOR ALL SSE Lengths.

newkeys = ['H1','H2','H3','H4','H5','H6','H7','H8','S1','S2','S3','S4','S5','S6','S7','S8','S9','S10','S11','S12','S13']

"""with open('adj.sse_lengths.tsv','w') as lenfile:
    lenfile.write("Element\tLength\n")
    for ki in['H8']:#newkeys:
        datarow = sse_lengths[ki]
        if "S" in ki: c = 'xkcd:orange'
        else: c = 'xkcd:denim'
        print(len(datarow))
        plot_hist(datarow, c, ki, all_base.length)
        val_string = f'{round(np.average(datarow),1)} ± {round(np.std(datarow),1)}'
        lenfile.write(f"{ki}\t{val_string}\n")

# Statistics for SSE residue conservation
with open("adj.sse_stats.tsv", "w") as statfile:
    statfile.write("Element\tOccupation\tConsensus\n")
    for sse in newkeys:
        datarow = center_res[f"{sse}.50"]
        occ, residues = parse_conservation(datarow, all_base.length)
        statfile.write(f"{sse}\t{occ}%\t{residues}\n")"""

In [ ]:
# Create the TSV file with the individual pLDDT values.
'''import re
identifiers = []
with open('all_plddt.tsv', 'w') as c:
    c.write('Identifier\tplddt_values\n')
    for j in jsons:
        identifier = re.findall(r'\/[\w]+-', j)[0][1:-1]
        if identifier in identifiers:
            continue
        identifiers.append(identifier)
        with open(j) as jx:
            data = json.load(jx)
        c.write(f"{identifier}\t{','.join(['{:.2f}'.format(k) for k in data['plddt']])}\n")'''
#print(len(identifiers))

In [ ]:
# Create a file where all pLDDT values are contained within, with each identifier
import json

jsons = glob.glob('/home/hildilab/projects/agpcr_nom/*_output/batch*/*rank_1*scores.json')

# construct a dictionary where for each identifier (i.e. H6.49), the total occ. is plottef

def construct_identifiers(indexing_dir, center_dir, plddt_values, max_id_dir, name, seq=None):
    id_dir = {}
    plddts = {}
    sse_seq = {}
    for sse in indexing_dir.keys():
        if sse == 'GPS' :
            continue
        start = indexing_dir[sse][0]
        end = indexing_dir[sse][1]
        if end-start > 45:
            print(f"NOTE: SKIPPDING TOO LONG SSE WITH LENGTH {end-start}\n{name}: {sse}")
            continue
        center_res = center_dir[f"{sse}.50"]
        first_res = 50 - center_res + start
        for k in range(end-start+1):
            if sse not in max_id_dir.keys():
                max_id_dir[sse] = []
            if first_res+k not in max_id_dir[sse]:
                max_id_dir[sse].append(first_res+k)
        id_dir[sse] = [first_res+k for k in range(end-start+1)]
        plddts[sse] = [plddt_values[k] for k in range(start, end+1)]
        if seq is not None:
            sse_seq[sse] = [seq[k] for k in range(start, end+1)]
    if seq is None:
        sse_seq = None
    return max_id_dir, id_dir, plddts, sse_seq

def get_plddt_dir(file='all_plddt.tsv'):
    plddt_dir = {}
    with open(file) as f:
        data = [l.strip() for l in f.readlines()[1:]]
        for l in data:
            i,v  = tuple(l.split("\t"))
            plddt_dir[i] = [float(val) for val in v.split(",")]
    return plddt_dir

def make_id_list(id_dir):
    id_list = []
    for sse in id_dir.keys():
        for res in id_dir[sse]:
            id_list.append(f"{sse}.{res}")
    return id_list #np.array(id_list)

def compact_label_positions(id_collection, plddt_collection, sse_keys):
    label_plddts = {}
    for sse in sse_keys:
        label_plddts[sse] = {}

    for i in range(len(id_collection)):
        gain_positions = id_collection[i]
        plddt_positions = plddt_collection[i]
        
        for sse in gain_positions.keys():
            
            for j, pos in enumerate(gain_positions[sse]):
                pos = int(pos)

                if pos not in label_plddts[sse].keys():
                    label_plddts[sse][pos] = [plddt_positions[sse][j]]
                else:
                    label_plddts[sse][pos].append(plddt_positions[sse][j])

    return label_plddts

def construct_id_occupancy(indexing_dirs, center_dirs, length, plddt_dir, names, seqs):
    newkeys = ['H1','H2','H3','H4','H5','H6','H7','H8','S1','S2','S3','S4','S5','S6','S7','S8','S9','S10','S11','S12','S13']
    id_collection = []
    plddt_collection = []
    seq_collection = []
    all_id_dir = {x:[] for x in newkeys}
    for k in range(length):
        identifier = names[k].split("-")[0]
        plddt_values = plddt_dir[identifier]
        all_id_dir, id_dir, plddts, sse_seq = construct_identifiers(indexing_dirs[k], center_dirs[k], plddt_values, all_id_dir, names[k], seqs[k])
        id_collection.append(id_dir)
        #print(id_dir)
        plddt_collection.append(plddts)
        seq_collection.append(sse_seq)
    print("Completed creating value collection.")
    print(id_collection[0])
    print(plddt_collection[0])

    # Here, parse through the id_dirs to count the occurrence of positions per SSE
    # Dictionary to map any label identifier to a respective position.
    id_map = {}
    i = 0
    for sse in newkeys:
        for res in all_id_dir[sse]:
            id_map[f'{sse}.{res}'] = i 
            i += 1
    
    max_id_list = []
    for i, id_dict in enumerate(id_collection):
        max_id_list.append(make_id_list(id_dict))
    flat_id_list = np.array([item for sublist in max_id_list for item in sublist])
    print("Finished constructing flat_id_list.")
    labels, occ = np.unique(flat_id_list, return_counts=True)
    # Parse through labels, occ to generate the sse-specific data
    occ_dict = {labels[u]:occ[u] for u in range(len(labels))}
    # Transform occ_dict to the same format as label_plddts (one dict per sse):
    label_occ = {}
    for sse in newkeys:
        label_occ[sse] = {int(k[-2:]):v for k,v in occ_dict.items() if sse in k}
    #print(labels, occ)
    label_plddts = compact_label_positions(id_collection, plddt_collection, newkeys)
    label_seq = compact_label_positions(id_collection, seq_collection, newkeys)
    #print(labels)
    return label_plddts, label_occ, label_seq
    #[print(k, len(v)) for k,v in label_plddts.items()]

plddt_dir = get_plddt_dir()
#print(list(plddt_dir.keys())[:10])
plddt_values, occ_values, label_seq = construct_id_occupancy(all_base.indexing_dirs, all_base.center_dirs, all_base.length, plddt_dir, all_base.names, seqs)

In [ ]:
## PLOT THE POSITION OCCUPANCY AND THE AVERAGE PLDDT PER POSITION. with plddt_values, occ_values
newkeys = ['H1','H2','H3','H4','H5','H6','H7','H8','S1','S2','S3','S4','S5','S6','S7','S8','S9','S10','S11','S12','S13']
for sse in newkeys:
    # Transform the values first
    pp = plddt_values[sse]
    #print(occ_values[sse])
    av_pp = {k:np.average(np.array(v))/100 for k,v in pp.items()}
    #print(av_pp)
    norm_occ = {k:v/14435 for k,v in occ_values[sse].items()}
    xax = sorted(av_pp.keys())
    y_pp = [av_pp[x] for x in xax]
    y_occ = [norm_occ[x] for x in xax]
    norm_pp = np.array(y_pp)*np.array(y_occ)

    fig, ax = plt.subplots(figsize=[5,2])
    fig.set_facecolor('w')
    ax.xaxis.set_minor_locator(MultipleLocator(1)) #AutoMinorLocator())
    ax.xaxis.set_major_locator(FixedLocator([a for a in range(2,100,3)]))#MultipleLocator(3)))
    ax.tick_params(which='both', width=2)
    ax.tick_params(which='major', length=8)
    ax.tick_params(which='minor', length=6)
    plt.bar(xax,y_pp, color='silver', alpha=0.7)
    plt.plot(xax, y_occ, color='dodgerblue')
    plt.bar(xax, norm_pp, color='xkcd:lightish red', alpha=0.1)
    plt.title(f'Element Composition ({sse})')
    plt.yticks(ticks = [0, 0.2, 0.4, 0.6, 0.8, 1], labels = ['0%', '20%', '40%', '60%', '80%', '100%'])
    #plt.ylabel('')
    ax.set_xticklabels([f'{sse}.{str(int(v))}' for v in ax.get_xticks()], rotation=90)
    #plt.savefig(f'../fig/{sse}_stats.svg', bbox_inches='tight')
    plt.show()
    plt.close(fig)

In [ ]:
# Get the occupancy of certain positions:
enriched_positions = ['H1.50','H1.54','H1.57','H1.61','H2.56','H2.57','H2.60','H2.61','H3.36','H3.43',
'H3.44','H3.51','H3.53','H3.56','H4.38','H4.41','H4.51','H5.37','H5.38','H5.42','H5.44',
'H5.48','H5.50','H5.59','H6.42','H6.54','H6.56','H7.40','H7.51','H8.46','H8.58','H8.60',
'S1.48','S2.47','S2.51','S2.53','S2.58','S3.53','S3.55','S4.56','S5.48','S5.52','S5.55',
'S6.50','S7.45','S7.52','S7.57','S9.53','S10.50','S11.54','S11.55','S12.47','S12.50','S12.52','S13.48','S13.50']
for sse in newkeys:
    sub_positions = [k for k in enriched_positions if f'{sse}.' in k]
    # Transform the values first
    #pp = plddt_values[sse]
    #print(occ_values[sse])
    #av_pp = {k:np.average(np.array(v))/100 for k,v in pp.items()}
    #print(av_pp)
    norm_occ = {f'{sse}.{k}':v/14435 for k,v in occ_values[sse].items()}
    #print(norm_occ)
    for k in sub_positions:
        print(round(norm_occ[k],2),k)


In [ ]:
# GENERATE A FULL DATAFRAME FOR THE LABELED POSITIONS AND THEIR RESPECTIVE AA FREQUENCIES FOR LOGOPLOTS
sse_aa_freqs = {}
aastr = 'ACDEFGHIKLMNPQRSTVWYX'
cols = {aa:i for i,aa in enumerate(aastr)}
for sse in newkeys:
    sse_dict = label_seq[sse]
    aafreqs = np.zeros(shape=(len(sse_dict.keys()), 21))
    for p_index, pos in enumerate(sorted(sse_dict.keys())):
        aas, freq = np.unique(np.array(sse_dict[pos]), return_counts=True)
        for i, aa in enumerate(aas):
            aafreqs[p_index, cols[aa]] = freq[i]/14435
    sse_aa_freqs[sse] = aafreqs


In [ ]:
# Print the sequence composition for each cancer-enriched-position

for sse in newkeys:
    sub_positions = [k for k in enriched_positions if f'{sse}.' in k]
    lframe = pd.DataFrame(data=sse_aa_freqs[sse], columns=[c for c in aastr], index = sorted(plddt_values[sse].keys()))
    #print(lframe)
    for pos in sub_positions:
        idx = int(pos[-2:]) # get the row number in the SSE
        res_data = lframe.loc[[idx]]
        total_freqs = res_data.sum(axis=1).to_list()[0]
        #print(f"{total_freqs = }, {type(total_freqs)}")

        norm_freq_dict = {round(freq.to_list()[0]/total_freqs, 5) : aa for aa, freq in res_data.items()}
        sorted_norm_freqs = sorted(norm_freq_dict.keys())[::-1]
        #print(pos)
        xstring = ''
        for k in sorted_norm_freqs[:3]: 
            xstring = xstring +f'{norm_freq_dict[k]}:{round(k*100)}%,'
        print(pos, xstring)

            # normalize frequency with the total sum

"""for sse in newkeys:
    for pos in enriched_positions:
        datarow = center_res[pos]
        occ, residues = parse_conservation(datarow, all_base.length)
        print(f"{sse}\t{occ}%\t{residues}\n")"""

In [ ]:
# LOGOPLOTS FOR THE ELEMENTS

from matplotlib.ticker import FixedLocator

for sse in newkeys:

    lframe = pd.DataFrame(data=sse_aa_freqs[sse], columns=[c for c in aastr], index = sorted(plddt_values[sse].keys()))

    # Note down the first and last row where the occupation threshold is met.
    firstval = None
    for i, r in lframe.iterrows():
        if np.sum(r) > 0.05: 
            if firstval is None:
                firstval = i
            lastval = i
    print(firstval, lastval)
    subframe = lframe.truncate(before=firstval, after=lastval)
    #x_offset = sorted(plddt_values[sse].keys())[0]

    fig, ax = plt.subplots(figsize=[5,2])
    cons_logo = logomaker.Logo(subframe,
                                ax=ax,
                                color_scheme='chemistry',
                                show_spines=False,
                                font_name='DejaVu Sans Mono')

    fig.set_facecolor('w')
    ax.xaxis.set_minor_locator(MultipleLocator(1)) #AutoMinorLocator())
    ax.xaxis.set_major_locator(FixedLocator([a for a in range(2,100,3)]))#MultipleLocator(3))
    ax.tick_params(which='both', width=2)
    ax.tick_params(which='major', length=8)
    ax.tick_params(which='minor', length=6)
    ax.set_xticklabels([f'{sse}.{str(int(v))}' for v in ax.get_xticks()], rotation=90)
    cons_logo.draw()
    fig.tight_layout()
    fig.set_facecolor('w')
    #plt.savefig(f"../fig/conslogo_{sse}.svg", bbox_inches='tight')
    plt.close(fig)

In [ ]:
for i, gain in enumerate(human_collection.collection):
    #if "Q6QNK2" in gain.name:
        x1, x2,_,_ = gain.create_indexing(anchors, anchor_occupation, anchor_dict, 
                    outdir = "/home/hildilab/projects/agpcr_nom/human_31/indexing_files_s2_dsp",
                    #offset = fasta_offsets[i]-gain.start+1,
                    split_mode='double')
        
        print(x1, x2)

In [ ]:
# A CELL FOR SPECIFICALLY EVALUATING INDIVIDUAL GAIN DOMAINS.
# MOSTLY FOR DEBUGGING OR CHECKING OUT ANOMALITIES
#[('G1SHT5-G1SHT5_RABIT-AGRG6-Oryctolagus_cuniculus', 'REANEVASEILNLTADGQNLTSANITSIVEQVKRIVNKEENIDVTLGSTLMNIFSNILSNSDSDLLESSSEALKTIDELAFKIDLNSTPHVNIATRNLALGVSSVSPGTNVISNFSIGLPSNNESYFQMDFESGQVDPLASVILPPNLLENLSQEDSILVRRAQFTFFNKTGLFQDVGPQRKTLVSYVMACSIGNITIQDLKDPVQIKIKHTRTQEVHHPICAFWDLNKNKSFGGWNTSGCIAHRDSDASETICLCNHFTHFGVLMD')
# ('G1SQ79-G1SQ79_RABIT-AGRG4-Oryctolagus_cuniculus', 'LQGLPDKILDLANITVSDENANDVAEHILNLLNESPPLDEEETKIIVSKVSDISLCEKISMNLTQLMLQIINAVLEKQNDSASGLHEVSNEILRLIERAGHKMEFWGRTANLMVARLALAMLRVDHKFEGVTFSIRSYTEGTDPEIYLGDVPAGKVLASIYLPKSLKKRLRVNNLQTILFNFFGQTSLFKVKNVSKALTTYVVSASISDLSIQNLADPVVITLQHVEGSQKYDQVHCAFWDFEKNNGLGGWNSSGCKVKETNVNYTICQCDHLTHFGVLMDL')
# ('G1T5U9-G1T5U9_RABIT-AGRL2-Oryctolagus_cuniculus'
tar = ["A0A151X191"]
tar_seqs = filter_by_list(valid_seqs, tar)
print(tar_seqs)
Xalignment_file = "/home/hildilab/projects/agpcr_nom/big_mafft14792.fa" # This is a combined alignment of ALL sequences in ALL queries!
Xquality_file = "/home/hildilab/projects/agpcr_nom/big_mafft14792.jal"  # ^ corresponding quality file
Xquality = sse_func.read_quality(Xquality_file)
Xgps_minus_one = 19258 #appended_big_mafft: 21160  #big_mafft14792: 19258
Xaln_cutoff = 19822 #appended_big_mafft: 21813 #big_mafft14792: 19822
Xstride_files = glob.glob("/home/hildilab/projects/agpcr_nom/all_gps_stride/*")
Xalignment_dict = sse_func.read_alignment(alignment_file, aln_cutoff)
Xname, Xseq = tar_seqs[0]
tar_stride = "/home/hildilab/projects/agpcr_nom/all_gps_stride/A0A369RSM9-A0A369RSM9_9METZ-AGRL3-Trichoplax_sp._H2.stride"
tar_gain = GainDomain(alignment_file = Xalignment_file,
                                        aln_cutoff = Xaln_cutoff,
                                        quality = Xquality,
                                        gps_index = Xgps_minus_one,
                                        name=Xname,
                                        #stride_files = Xstride_files,
                                        sequence = Xseq,
                                        alignment_dict = Xalignment_dict,
                                        explicit_stride_file=tar_stride,
                                        is_truncated = True,
                                        coil_weight = 0.01, # TESTING
                                        stride_outlier_mode = True,
                                        without_anchors=True)
#print(tar_gain.isValid)
#if tar_gain.isValid: 
#    tar_gain.plot_helicality(savename=f"{tar_gain.name.split('-')[0]}_helicality.full.svg", debug=True)
#    tar_gain.plot_profile()
#print(tar_gain.subdomain_boundary, tar_gain.start, tar_gain.end)
    #print(gain.name, gain.start, gain.end, gain.sequence, gain.index, gain.subdomain_boundary)
    #pdb_out = root_path+"human_32/trunc_pdbs/"+gain.name+"_gain.pdb"
    #ac = gain.name.split("-")[0]
    #found_pdb = [pdb for pdb in pdb_list if ac in pdb]
    #target_pdb = found_pdb[0]
    #gain.write_gain_pdb(target_pdb, pdb_out) """


In [ ]:
aln_col = []

for idx, idx_dir in enumerate(all_base.indexing_dirs):
    try:
        l = idx_dir['H6']
    except:
        continue
    print(l)
    
    for i in range(l[0], l[1]+1):
        if valid_collection.collection[idx].sequence[i] == 'W':
            aln_col.append(valid_collection.collection[idx].alignment_indices[i])
            print(valid_collection.collection[idx].sequence[i],valid_collection.collection[idx].alignment_indices[i])
    
print(len(aln_col), np.unique(np.array(aln_col), return_counts=True))


In [ ]:
# STATISTICS FOR ELEMENT-CONNECTING LOOPS
[print(v, len(k), round(np.average(k),2), round(np.std(k),2)) for v,k in loop_lengths.items()]

In [ ]:
def get_loop_stats(indexing_dir, sequence):
    # Returns a named dict with loop lengths, i.e. {"H1-H2":13, "H8-S1":12}
    inverted_dir = {sse[0] : (sse[1],ki) for ki, sse in indexing_dir.items() if "GPS" not in ki} # The begin of each sse is here {0:(13, "H2")}
    loop_loc = {}
    loop_dir = {}
    ordered_starts = sorted(inverted_dir.keys())
    for i, sse_start in enumerate(ordered_starts):
        if i == 0: 
            continue # Skip the first and go from the second SSE onwards, looking in N-terminal direction.
        c_label = inverted_dir[sse_start][1]
        n_end, n_label = inverted_dir[ordered_starts[i-1]]
        loop_loc[f"{n_label}-{c_label}"] = (n_end, sse_start-1)
        loop_dir[f"{n_label}-{c_label}"] = sequence[n_end+1:sse_start] # The one-letter-coded seqeuence. Will be a list of lists
    return loop_loc, loop_dir

loop_lengths = {}
loop_seqs = {}
loop_seq = {}

loop_info = {}
#[loop_info[loop] = {} for loop in loop_seqs.keys()] # into each of these keys, any entry is composed of "name":$name, "sequence":$seq

for idx in range(all_base.length):
    curr_name = all_base.names[idx]
    start = valid_collection.collection[idx].start
    i_loc, i_dir = get_loop_stats(all_base.indexing_dirs[idx], valid_collection.collection[idx].sequence)
    for k, seq in i_dir.items():
        if k not in loop_info.keys():
            loop_info[k] = []
        loop_info[k].append({'name':f'{all_base.names[idx]}_{i_loc[k][0]+start}-{i_loc[k][1]+start}', 'sequence':''.join(seq)})
    #print(i_len)
    #loop_lengths = match_dirs(i_len, loop_lengths)
    #loop_seqs = match_dirs(i_dir, loop_seqs)



In [ ]:
# Write the collected loop sequences to a FASTA file for later alignment.
def loop2fasta(outfile, itemlist):
    with open(outfile, 'w') as out:
        for subdict in itemlist:
            out.write(f">{subdict['name']}\n{subdict['sequence']}\n")
    print("Done with", outfile)

for loop in loop_info.keys():
    loop2fasta(f"../loops/{loop}.fa", loop_info[loop])


In [ ]:
# Get all structures containing 7 Helices in Subdomain A.

mystring = ''
no=0
for gain in valid_collection.collection:
        try:sse_labl, _, _, _ = gain.create_indexing(silent=True,anchors=anchors, anchor_occupation=anchor_occupation, anchor_dict=anchor_dict, debug=False)
        except:no+=1;continue
        kk = len([k for k in sse_labl.keys() if "H" in k])
        if kk >= 7: mystring += gain.name+"\n"#+str(kk)+"\n"
print(no)
print(mystring)
with open("7helix.txt", 'w') as seven:
        seven.write(mystring)

In [ ]:
print(len(mystring.split("\n")))

In [ ]:
# Get all structures containing 14+ Strands in Subdomain A.

mystring = ''
no=0
num_sheets = np.zeros(shape=(14432))
total_strand = np.zeros(shape=(14432))
for i, gain in enumerate(valid_collection.collection):
        #try:sse_labl, _, _, _ = gain.create_indexing(silent=True,anchors=anchors, anchor_occupation=anchor_occupation, anchor_dict=anchor_dict, debug=False)
        #except:no+=1;continue
        num_sheets[i] = len(gain.sdb_sheets)
        if len(gain.sdb_sheets) > 14 and len(gain.sdb_sheets) < 17:
                mystring += gain.name+"\n"#+str(kk)+"\n"

        total_strand[i] = np.sum([strand[1]-strand[0] for strand in gain.sdb_sheets])
        #k = len([k for k in sse_labl.keys() if "S" in k])
        #if kk >= 13: mystring += gain.name+"\n"#+str(kk)+"\n"
print(no)
print(np.unique(num_sheets, return_counts=True))

#print(np.unique(total_strand, return_counts=True, return_index=True))
with open("14plussheet.txt", 'w') as seven:
        seven.write(mystring)

In [ ]:
print(valid_collection.collection[11476].name)

In [ ]:
extents = {}
for gain in valid_collection.collection:
    extents[gain.name.split("-")[0]] = [str(gain.start+1), str(gain.subdomain_boundary+1), str(gain.end+1)] # make it compatible with ONE-indexed PDBs.

import json
with open('domain_extents.json', 'w') as j:
    dump = json.dumps(extents)
    j.write(dump)
